In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch_geometric.data import Data

In [42]:
body_df = pd.read_csv("../data/Reddit/raw/soc-redditHyperlinks-body.tsv",sep="\t")
body_df

,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,POST_ID,TIMESTAMP,LINK_SENTIMENT,PROPERTIES
0,leagueoflegends,teamredditteams,1u4nrps,2013-12-31 16:39:58,1,"345.0,298.0,0.75652173913,0.0173913043478,0.08..."
1,theredlion,soccer,1u4qkd,2013-12-31 18:18:37,-1,"101.0,98.0,0.742574257426,0.019801980198,0.049..."
2,inlandempire,bikela,1u4qlzs,2014-01-01 14:54:35,1,"85.0,85.0,0.752941176471,0.0235294117647,0.082..."
3,nfl,cfb,1u4sjvs,2013-12-31 17:37:55,1,"1124.0,949.0,0.772241992883,0.0017793594306,0...."
4,playmygame,gamedev,1u4w5ss,2014-01-01 02:51:13,1,"715.0,622.0,0.777622377622,0.00699300699301,0...."
...,...,...,...,...,...,...
286556,negareddit,debatefascism,68im20s,2017-04-30 16:31:26,1,"441.0,405.0,0.775510204082,0.0294784580499,0.0..."
286557,mildlynomil,justnomil,68imlas,2017-04-30 04:19:03,1,"2226.0,1855.0,0.786163522013,0.00224618149146,..."
286558,mmorpg,blackdesertonline,68ip5os,2017-04-30 16:54:08,1,"1100.0,909.0,0.778181818182,0.00181818181818,0..."
286559,electricskateboards,askreddit,68ipb2s,2017-04-30 16:41:53,1,"1876.0,1567.0,0.78144989339,0.00692963752665,0..."


In [43]:
title_df = pd.read_csv("../data/Reddit/raw/soc-redditHyperlinks-title.tsv",sep='\t')
title_df

,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,POST_ID,TIMESTAMP,LINK_SENTIMENT,PROPERTIES
0,rddtgaming,rddtrust,1u4pzzs,2013-12-31 16:39:18,1,"25.0,23.0,0.76,0.0,0.44,0.12,0.12,4.0,4.0,0.0,..."
1,xboxone,battlefield_4,1u4tmfs,2013-12-31 17:59:11,1,"100.0,88.0,0.78,0.02,0.08,0.13,0.07,16.0,16.0,..."
2,ps4,battlefield_4,1u4tmos,2013-12-31 17:59:40,1,"100.0,88.0,0.78,0.02,0.08,0.13,0.07,16.0,16.0,..."
3,fitnesscirclejerk,leangains,1u50xfs,2013-12-31 19:01:56,1,"49.0,43.0,0.775510204082,0.0,0.265306122449,0...."
4,fitnesscirclejerk,lifeprotips,1u51nps,2013-12-31 21:02:28,1,"14.0,14.0,0.785714285714,0.0,0.428571428571,0...."
...,...,...,...,...,...,...
571922,seattleshitshow,seattlewa,68iptp,2017-04-27 11:07:31,-1,"83.0,70.0,0.78313253012,0.0,0.10843373494,0.16..."
571923,peoplewhosayheck,spiderman,68iqh6s,2017-04-30 16:36:07,1,"121.0,105.0,0.719008264463,0.0165289256198,0.0..."
571924,vzla,wtf,68iqkcs,2017-04-30 15:50:07,1,"68.0,58.0,0.764705882353,0.0,0.147058823529,0...."
571925,denverbroncos,nfl,68iqt8,2017-04-30 16:11:30,-1,"96.0,79.0,0.770833333333,0.0,0.0729166666667,0..."


In [44]:
subreddit_embedding_df = pd.read_csv('../data/Reddit/raw/web-redditEmbeddings-subreddits.csv',header=None)
embedding_cols = [col for col in subreddit_embedding_df.columns[1:]]
subreddit_embedding_df['embedding'] = subreddit_embedding_df[embedding_cols].values.tolist()
subreddit_embedding_df = subreddit_embedding_df.drop(columns=embedding_cols)
subreddit_embedding_df.columns.values[0] = 'node_id'
subreddit_embedding_df

,node_id,embedding
0,spiders,"[0.158972, 0.285813, 0.226329, -0.183338, -0.1..."
1,askreddit,"[-0.499114, 0.323983, -0.424809, -0.222705, -0..."
2,globaloffensivetrade,"[-0.023145, -1.199374, 1.661484, -1.025296, 1...."
3,fireteams,"[2.492506, -2.529917, -0.448484, -3.543441, -0..."
4,funny,"[-0.81937, -0.865261, 0.301753, 0.018787, 0.20..."
...,...,...
51273,33rd,"[-7.6e-05, -0.001643, -0.002896, 0.000581, -0...."
51274,bestofvic2015,"[-0.000203, 0.012698, -0.007575, 0.006486, -0...."
51275,aberystwyth,"[-0.001563, 0.004733, -8.2e-05, -0.000982, -0...."
51276,mail_forwarding,"[0.009269, 0.024779, -0.017018, -0.007228, 0.0..."


In [45]:
node_id_to_index = {nid:i for i,nid in enumerate(subreddit_embedding_df['node_id'])}
print(len(node_id_to_index))

51278


In [46]:
body_df_ids = pd.unique(body_df[['SOURCE_SUBREDDIT', 'TARGET_SUBREDDIT']].values.ravel())

In [47]:
node_id_to_index = {nid:i for i,nid in enumerate(body_df_ids)}

In [48]:
len(node_id_to_index)

35776

## Preparing reddit body

In [49]:
body_subreddit_name_df = pd.DataFrame(pd.unique(body_df[['SOURCE_SUBREDDIT', 'TARGET_SUBREDDIT']].values.ravel()),columns=['node_id'])
body_subreddit_name_df

,node_id
0,leagueoflegends
1,teamredditteams
2,theredlion
3,soccer
4,inlandempire
...,...
35771,wetclothes
35772,daddyofive
35773,hatedaddyofive
35774,noshitouija


In [50]:
node_embeddings = body_subreddit_name_df.merge(subreddit_embedding_df,
                                                    left_on='node_id',
                                                    right_on='node_id',
                                                    how='left')

In [51]:
node_embeddings

,node_id,embedding
0,leagueoflegends,"[-2.785298, -0.166391, 1.592624, -1.269829, 2...."
1,teamredditteams,"[-1.402933, -1.115654, 0.921651, -0.711803, 1...."
2,theredlion,"[-0.239977, -0.246483, 0.059679, -0.019758, -0..."
3,soccer,"[-2.798847, -1.243099, 0.513319, 0.162192, 1.3..."
4,inlandempire,"[0.070123, -0.001389, 0.143414, -0.466989, -0...."
...,...,...
35771,wetclothes,NaN
35772,daddyofive,NaN
35773,hatedaddyofive,NaN
35774,noshitouija,NaN


In [52]:
def random_embedding():
    return ((np.random.rand(300) * 1e5).astype(int) / 1e5).tolist()
node_embeddings['embedding'] = node_embeddings['embedding'].apply(lambda x: random_embedding() if isinstance(x, float) and pd.isna(x) else x)

In [13]:
node_embeddings

,node_id,embedding
0,leagueoflegends,"[-2.785298, -0.166391, 1.592624, -1.269829, 2...."
1,teamredditteams,"[-1.402933, -1.115654, 0.921651, -0.711803, 1...."
2,theredlion,"[-0.239977, -0.246483, 0.059679, -0.019758, -0..."
3,soccer,"[-2.798847, -1.243099, 0.513319, 0.162192, 1.3..."
4,inlandempire,"[0.070123, -0.001389, 0.143414, -0.466989, -0...."
...,...,...
35771,wetclothes,"[0.03079, 0.65988, 0.83901, 0.42406, 0.98429, ..."
35772,daddyofive,"[0.99885, 0.85992, 0.1248, 0.02335, 0.71713, 0..."
35773,hatedaddyofive,"[0.96754, 0.60068, 0.40072, 0.81495, 0.23656, ..."
35774,noshitouija,"[0.48956, 0.39216, 0.43982, 0.42061, 0.01223, ..."


In [14]:
node_id_to_index = {nid:i for i,nid in enumerate(node_embeddings['node_id'])}
print(len(node_id_to_index))

35776


In [15]:
node_embeddings['node_index'] = node_embeddings['node_id'].map(node_id_to_index)
node_embeddings

,node_id,embedding,node_index
0,leagueoflegends,"[-2.785298, -0.166391, 1.592624, -1.269829, 2....",0
1,teamredditteams,"[-1.402933, -1.115654, 0.921651, -0.711803, 1....",1
2,theredlion,"[-0.239977, -0.246483, 0.059679, -0.019758, -0...",2
3,soccer,"[-2.798847, -1.243099, 0.513319, 0.162192, 1.3...",3
4,inlandempire,"[0.070123, -0.001389, 0.143414, -0.466989, -0....",4
...,...,...,...
35771,wetclothes,"[0.03079, 0.65988, 0.83901, 0.42406, 0.98429, ...",35771
35772,daddyofive,"[0.99885, 0.85992, 0.1248, 0.02335, 0.71713, 0...",35772
35773,hatedaddyofive,"[0.96754, 0.60068, 0.40072, 0.81495, 0.23656, ...",35773
35774,noshitouija,"[0.48956, 0.39216, 0.43982, 0.42061, 0.01223, ...",35774


In [28]:
body_df['source_idx'] = body_df['SOURCE_SUBREDDIT'].map(node_id_to_index)
body_df['target_idx'] = body_df['TARGET_SUBREDDIT'].map(node_id_to_index)
body_df

,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,POST_ID,TIMESTAMP,LINK_SENTIMENT,PROPERTIES,src_id,dest_id,datetime,time_group,source_idx,target_idx
0,leagueoflegends,teamredditteams,1u4nrps,2013-12-31 16:39:58,1,"345.0,298.0,0.75652173913,0.0173913043478,0.08...",0,1,2013-12-31 16:39:58,2013-12-30/2014-01-05,0,1
1,theredlion,soccer,1u4qkd,2013-12-31 18:18:37,-1,"101.0,98.0,0.742574257426,0.019801980198,0.049...",2,3,2013-12-31 18:18:37,2013-12-30/2014-01-05,2,3
2,inlandempire,bikela,1u4qlzs,2014-01-01 14:54:35,1,"85.0,85.0,0.752941176471,0.0235294117647,0.082...",4,5,2014-01-01 14:54:35,2013-12-30/2014-01-05,4,5
3,nfl,cfb,1u4sjvs,2013-12-31 17:37:55,1,"1124.0,949.0,0.772241992883,0.0017793594306,0....",6,7,2013-12-31 17:37:55,2013-12-30/2014-01-05,6,7
4,playmygame,gamedev,1u4w5ss,2014-01-01 02:51:13,1,"715.0,622.0,0.777622377622,0.00699300699301,0....",8,9,2014-01-01 02:51:13,2013-12-30/2014-01-05,8,9
...,...,...,...,...,...,...,...,...,...,...,...,...
286556,negareddit,debatefascism,68im20s,2017-04-30 16:31:26,1,"441.0,405.0,0.775510204082,0.0294784580499,0.0...",1604,3023,2017-04-30 16:31:26,2017-04-24/2017-04-30,1604,3023
286557,mildlynomil,justnomil,68imlas,2017-04-30 04:19:03,1,"2226.0,1855.0,0.786163522013,0.00224618149146,...",35074,18935,2017-04-30 04:19:03,2017-04-24/2017-04-30,35074,18935
286558,mmorpg,blackdesertonline,68ip5os,2017-04-30 16:54:08,1,"1100.0,909.0,0.778181818182,0.00181818181818,0...",836,14717,2017-04-30 16:54:08,2017-04-24/2017-04-30,836,14717
286559,electricskateboards,askreddit,68ipb2s,2017-04-30 16:41:53,1,"1876.0,1567.0,0.78144989339,0.00692963752665,0...",29652,59,2017-04-30 16:41:53,2017-04-24/2017-04-30,29652,59


In [39]:
body_df['label'] = body_df['LINK_SENTIMENT'].map({-1:1,1:1})

In [34]:
def parse_feature_string(feat_str):
    return [float(f) for f in feat_str.split(',')]
body_df['edge_features'] = body_df['PROPERTIES'].apply(parse_feature_string)

In [35]:
body_df["datetime"] = pd.to_datetime(body_df["TIMESTAMP"])
body_df['time_group'] = body_df['datetime'].dt.to_period('W')
grouped_body_df = body_df.groupby('time_group')
body_df

,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,POST_ID,TIMESTAMP,LINK_SENTIMENT,PROPERTIES,src_id,dest_id,datetime,time_group,source_idx,target_idx,edge_features
0,leagueoflegends,teamredditteams,1u4nrps,2013-12-31 16:39:58,1,"345.0,298.0,0.75652173913,0.0173913043478,0.08...",0,1,2013-12-31 16:39:58,2013-12-30/2014-01-05,0,1,"[345.0, 298.0, 0.75652173913, 0.0173913043478,..."
1,theredlion,soccer,1u4qkd,2013-12-31 18:18:37,-1,"101.0,98.0,0.742574257426,0.019801980198,0.049...",2,3,2013-12-31 18:18:37,2013-12-30/2014-01-05,2,3,"[101.0, 98.0, 0.742574257426, 0.019801980198, ..."
2,inlandempire,bikela,1u4qlzs,2014-01-01 14:54:35,1,"85.0,85.0,0.752941176471,0.0235294117647,0.082...",4,5,2014-01-01 14:54:35,2013-12-30/2014-01-05,4,5,"[85.0, 85.0, 0.752941176471, 0.0235294117647, ..."
3,nfl,cfb,1u4sjvs,2013-12-31 17:37:55,1,"1124.0,949.0,0.772241992883,0.0017793594306,0....",6,7,2013-12-31 17:37:55,2013-12-30/2014-01-05,6,7,"[1124.0, 949.0, 0.772241992883, 0.001779359430..."
4,playmygame,gamedev,1u4w5ss,2014-01-01 02:51:13,1,"715.0,622.0,0.777622377622,0.00699300699301,0....",8,9,2014-01-01 02:51:13,2013-12-30/2014-01-05,8,9,"[715.0, 622.0, 0.777622377622, 0.0069930069930..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
286556,negareddit,debatefascism,68im20s,2017-04-30 16:31:26,1,"441.0,405.0,0.775510204082,0.0294784580499,0.0...",1604,3023,2017-04-30 16:31:26,2017-04-24/2017-04-30,1604,3023,"[441.0, 405.0, 0.775510204082, 0.0294784580499..."
286557,mildlynomil,justnomil,68imlas,2017-04-30 04:19:03,1,"2226.0,1855.0,0.786163522013,0.00224618149146,...",35074,18935,2017-04-30 04:19:03,2017-04-24/2017-04-30,35074,18935,"[2226.0, 1855.0, 0.786163522013, 0.00224618149..."
286558,mmorpg,blackdesertonline,68ip5os,2017-04-30 16:54:08,1,"1100.0,909.0,0.778181818182,0.00181818181818,0...",836,14717,2017-04-30 16:54:08,2017-04-24/2017-04-30,836,14717,"[1100.0, 909.0, 0.778181818182, 0.001818181818..."
286559,electricskateboards,askreddit,68ipb2s,2017-04-30 16:41:53,1,"1876.0,1567.0,0.78144989339,0.00692963752665,0...",29652,59,2017-04-30 16:41:53,2017-04-24/2017-04-30,29652,59,"[1876.0, 1567.0, 0.78144989339, 0.006929637526..."


In [40]:
x = torch.tensor(node_embeddings['embedding'].tolist(), dtype=torch.float)
data_list = []
for _, group in grouped_body_df:
    data = Data()
    data.x = x
    data.edge_index = torch.tensor(group[['source_idx', 'target_idx']].values.T, dtype=torch.long)
    data.y = torch.tensor(group['label'].values, dtype=torch.float)
    data.edge_attr = torch.tensor(group['edge_features'].tolist(),
                                          dtype=torch.float)
    print(data)

Data(x=[35776, 300], edge_index=[2, 500], y=[500], edge_attr=[500, 86])
Data(x=[35776, 300], edge_index=[2, 961], y=[961], edge_attr=[961, 86])
Data(x=[35776, 300], edge_index=[2, 1114], y=[1114], edge_attr=[1114, 86])
Data(x=[35776, 300], edge_index=[2, 1034], y=[1034], edge_attr=[1034, 86])
Data(x=[35776, 300], edge_index=[2, 1108], y=[1108], edge_attr=[1108, 86])
Data(x=[35776, 300], edge_index=[2, 1073], y=[1073], edge_attr=[1073, 86])
Data(x=[35776, 300], edge_index=[2, 1218], y=[1218], edge_attr=[1218, 86])
Data(x=[35776, 300], edge_index=[2, 1090], y=[1090], edge_attr=[1090, 86])
Data(x=[35776, 300], edge_index=[2, 1065], y=[1065], edge_attr=[1065, 86])
Data(x=[35776, 300], edge_index=[2, 1144], y=[1144], edge_attr=[1144, 86])
Data(x=[35776, 300], edge_index=[2, 1358], y=[1358], edge_attr=[1358, 86])
Data(x=[35776, 300], edge_index=[2, 1116], y=[1116], edge_attr=[1116, 86])
Data(x=[35776, 300], edge_index=[2, 1275], y=[1275], edge_attr=[1275, 86])
Data(x=[35776, 300], edge_index